In [1]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw

In [2]:
image_dir = "/home/noob/koty/new_before_last/project-ano/shit"
mask_dir = "/home/noob/koty/new_before_last/project-ano/masks-shit"

In [3]:
os.makedirs(mask_dir, exist_ok=True)

In [4]:
class_mapping = {
    "Eppendorf": 1,           # Class 1
    "empty_part": 2,          # Class 2
    "importat_liquid": 3      # Class 3
}

In [5]:
def create_multiclass_mask(json_path, class_mapping):
    """
    Convert LabelMe JSON annotations to multi-class segmentation masks
    """
    with open(json_path, "r") as f:
        data = json.load(f)
    
    # Get image dimensions
    width = data['imageWidth']
    height = data['imageHeight']
    
    # Create empty mask (background = 0)
    mask = np.zeros((height, width), dtype=np.uint8)
    
    # Track which classes were found
    found_classes = set()
    
    # Process each shape/annotation
    for shape in data["shapes"]:
        label = shape["label"]
        
        if label in class_mapping:
            class_id = class_mapping[label]
            found_classes.add(label)
            
            # Create temporary PIL image for drawing
            temp_mask = Image.new("L", (width, height), 0)
            draw = ImageDraw.Draw(temp_mask)
            
            # Convert points to tuples
            points = [tuple(point) for point in shape["points"]]
            
            # Draw filled polygon
            draw.polygon(points, outline=class_id, fill=class_id)
            
            # Convert to numpy and overlay on main mask
            temp_np = np.array(temp_mask)
            
            # Only update pixels where temp_mask is non-zero
            # This handles overlapping annotations (later ones take priority)
            mask[temp_np > 0] = temp_np[temp_np > 0]
        else:
            # Skip unknown labels - they remain as background (class 0)
            print(f"⚠️ Label '{label}' in {os.path.basename(json_path)} -> treating as background")
    
    return mask, found_classes


In [6]:
def save_mask_variants(mask, base_name, mask_dir):
    """
    Save mask in different formats
    """
    # Save raw mask (class IDs as pixel values)
    mask_path = os.path.join(mask_dir, f"{base_name}_mask.png")
    Image.fromarray(mask).save(mask_path)
    
    # Save colorized version for visualization
    colorized_mask = create_colorized_mask(mask)
    colorized_path = os.path.join(mask_dir, f"{base_name}_mask_colored.png")
    colorized_mask.save(colorized_path)
    
    return mask_path, colorized_path


In [7]:
def create_colorized_mask(mask):
    """
    Create a colorized version of the mask for better visualization
    """
    # Define colors for each class (RGB)
    colors = {
        0: (0, 0, 0),       # Background - Black
        1: (255, 0, 0),     # Class 1 - Red
        2: (0, 255, 0),     # Class 2 - Green
        3: (0, 0, 255),     # Class 3 - Blue
    }
    
    height, width = mask.shape
    colored_mask = np.zeros((height, width, 3), dtype=np.uint8)
    
    for class_id, color in colors.items():
        colored_mask[mask == class_id] = color
    
    return Image.fromarray(colored_mask)


In [8]:
def print_statistics(all_found_classes, total_files):
    """
    Print processing statistics
    """
    print(f"\n📊 Processing Statistics:")
    print(f"Total files processed: {total_files}")
    print(f"Classes found across all files: {sorted(all_found_classes)}")
    
    class_counts = {label: 0 for label in class_mapping.keys()}
    # Note: You might want to track per-file statistics for more detailed reporting


In [9]:
# Main processing loop
print(f"🚀 Starting multi-class mask conversion...")
print(f"Class mapping: {class_mapping}")
print(f"Input directory: {image_dir}")
print(f"Output directory: {mask_dir}")
print("-" * 50)


🚀 Starting multi-class mask conversion...
Class mapping: {'Eppendorf': 1, 'empty_part': 2, 'importat_liquid': 3}
Input directory: /home/noob/koty/new_before_last/project-ano/shit
Output directory: /home/noob/koty/new_before_last/project-ano/masks-shit
--------------------------------------------------


In [10]:
all_found_classes = set()
processed_files = 0

for filename in os.listdir(image_dir):
    if not filename.endswith(".json"):
        continue
    
    json_path = os.path.join(image_dir, filename)
    
    try:
        # Create multi-class mask
        mask, found_classes = create_multiclass_mask(json_path, class_mapping)
        
        # Update global statistics
        all_found_classes.update(found_classes)
        processed_files += 1
        
        # Get base name for output files
        with open(json_path, "r") as f:
            data = json.load(f)
        base_name = os.path.splitext(data["imagePath"])[0]
        
        # Save masks
        mask_path, colored_path = save_mask_variants(mask, base_name, mask_dir)
        
        # Print progress
        print(f"✅ {filename}")
        print(f"   Classes found: {sorted(found_classes) if found_classes else ['None']}")
        print(f"   Saved: {os.path.basename(mask_path)}")
        print(f"   Colored: {os.path.basename(colored_path)}")
        
        # Show class distribution in this mask
        unique_classes, counts = np.unique(mask, return_counts=True)
        class_stats = dict(zip(unique_classes, counts))
        print(f"   Class distribution: {class_stats}")
        print()
        
    except Exception as e:
        print(f"❌ Error processing {filename}: {str(e)}")
        continue

✅ 1.json
   Classes found: ['Eppendorf', 'empty_part', 'importat_liquid']
   Saved: 1_mask.png
   Colored: 1_mask_colored.png
   Class distribution: {np.uint8(0): np.int64(1585416), np.uint8(1): np.int64(8614), np.uint8(2): np.int64(11733), np.uint8(3): np.int64(32637)}

✅ 4.json
   Classes found: ['Eppendorf', 'importat_liquid']
   Saved: 4_mask.png
   Colored: 4_mask_colored.png
   Class distribution: {np.uint8(0): np.int64(1580577), np.uint8(1): np.int64(12234), np.uint8(3): np.int64(45589)}

✅ 3.json
   Classes found: ['Eppendorf', 'empty_part', 'importat_liquid']
   Saved: 3_mask.png
   Colored: 3_mask_colored.png
   Class distribution: {np.uint8(0): np.int64(1584129), np.uint8(1): np.int64(9325), np.uint8(2): np.int64(12681), np.uint8(3): np.int64(32265)}

✅ 5.json
   Classes found: ['Eppendorf', 'empty_part', 'importat_liquid']
   Saved: 5_mask.png
   Colored: 5_mask_colored.png
   Class distribution: {np.uint8(0): np.int64(1583247), np.uint8(1): np.int64(9486), np.uint8(2): np.

In [11]:
print_statistics(all_found_classes, processed_files)

print(f"\n🎉 Conversion complete!")
print(f"Raw masks saved with class IDs (0-3) as pixel values")
print(f"Colored masks saved for visualization")


📊 Processing Statistics:
Total files processed: 5
Classes found across all files: ['Eppendorf', 'empty_part', 'importat_liquid']

🎉 Conversion complete!
Raw masks saved with class IDs (0-3) as pixel values
Colored masks saved for visualization
